In [21]:
import pandas as pd 
from sklearn.model_selection import train_test_split 
from sklearn.preprocessing import StandardScaler , OneHotEncoder,LabelEncoder
import pickle

In [22]:
data = pd.read_csv('Data\Churn_Modelling.csv')
data.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [23]:
# Data Preprocessing
data.drop(['RowNumber','CustomerId','Surname'],axis=1,inplace=True)

In [24]:
# Encoding Categorical Data
labelencoder = LabelEncoder()
data['Gender']= labelencoder.fit_transform(data['Gender'])

In [25]:
# one hot encoding for categorical variable
ohe = OneHotEncoder()
geo_encoded = ohe.fit_transform(data[['Geography']]).toarray()
geo_encoded = pd.DataFrame(geo_encoded, columns=ohe.get_feature_names_out(['Geography']))
data = pd.concat([data, geo_encoded], axis=1)
data.drop(['Geography'], axis=1, inplace=True)


In [26]:

data.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0


In [27]:
# split the data into features and target 
X= data.drop(['EstimatedSalary'],axis=1)
y= data['EstimatedSalary']

In [28]:
# split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [29]:
# scale the features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)


In [30]:
# save the encoders and scaler for later use 
with open ('labelencoder2.pkl','wb') as file:
    pickle.dump(labelencoder,file)

with open ('ohe2.pkl','wb') as file:
    pickle.dump(ohe,file)


with open ('scaler2.pkl','wb') as file:
    pickle.dump(scaler,file)


In [31]:
## Ann regression problemstatement
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout


In [32]:
model = Sequential([
    Dense(64, activation = 'relu', input_shape=(X_train.shape[1],)),
    Dense(32, activation = 'relu'),
    Dense(1, activation = 'linear') # output layer for regression 
])

# compile the model 
model.compile(optimizer='adam', loss='mean_absolute_error', metrics=['mae'])

model.summary()



Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 64)                832       
                                                                 
 dense_1 (Dense)             (None, 32)                2080      
                                                                 
 dense_2 (Dense)             (None, 1)                 33        
                                                                 
Total params: 2945 (11.50 KB)
Trainable params: 2945 (11.50 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [33]:
from tensorflow.keras.callbacks import EarlyStopping , TensorBoard
import datetime

# set up Tensorboard 
log_dir = "regressionLOgs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)

In [ ]:
## set up Early Stopping 

early_stopping_callback = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

In [35]:
# train the model 
history = model.fit(X_train , y_train,
                    validation_data=(X_test, y_test),
                    epochs=100,
                    callbacks=[tensorboard_callback, early_stopping_callback])

Epoch 1/100


250/250 [==============================] - 2s 5ms/step - loss: 100377.9688 - mae: 100377.9688 - val_loss: 98518.6328 - val_mae: 98518.6328
Epoch 2/100
250/250 [==============================] - 1s 3ms/step - loss: 99642.7969 - mae: 99642.7969 - val_loss: 97045.4844 - val_mae: 97045.4844
Epoch 3/100
250/250 [==============================] - 1s 3ms/step - loss: 97080.2812 - mae: 97080.2812 - val_loss: 93296.8828 - val_mae: 93296.8828
Epoch 4/100
250/250 [==============================] - 1s 3ms/step - loss: 92022.0625 - mae: 92022.0625 - val_loss: 87013.5078 - val_mae: 87013.5078
Epoch 5/100
250/250 [==============================] - 1s 3ms/step - loss: 84637.9531 - mae: 84637.9531 - val_loss: 78955.5625 - val_mae: 78955.5625
Epoch 6/100
250/250 [==============================] - 1s 3ms/step - loss: 75906.9297 - mae: 75906.9297 - val_loss: 70343.2734 - val_mae: 70343.2734
Epoch 7/100
250/250 [==============================] - 1s 5ms/step - loss: 67342.2969 - mae: 67342.296

In [36]:
%load_ext tensorboard


In [38]:
%tensorboard --logdir regressionLOgs/fit

Reusing TensorBoard on port 6006 (pid 15948), started 0:00:05 ago. (Use '!kill 15948' to kill it.)

In [39]:
## Evaluate the model
loss, mae = model.evaluate(X_test, y_test)
print(f"Test Loss: {loss}, Test MAE: {mae}")


63/63 [==============================] - 0s 2ms/step - loss: 50293.2656 - mae: 50293.2656
Test Loss: 50293.265625, Test MAE: 50293.265625


In [40]:
# save the model 

model.save('salary_regression_model.h5')

d:\365 projects\Ann classification\ANN-Classification-Project\Ann\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
